In [240]:
import pandas as pd
import numpy as np
from sklearn.linear_model import *
from sklearn.datasets import *
from sklearn.metrics import *
from sklearn.preprocessing import *

def d(data):
    return [ "%.2f" % i for i in data ]

def evaluate(coefs, x, y, logit=False):
    y_pred = np.dot(x, coefs)
    if(logit):
        y_pred = sc.special.expit(y_pred)
        return "LL %.3f / AUC %.3f" % ( log_loss(y, y_pred), roc_auc_score(y, y_pred))
    return "MSE %.3f / R2 %.3f" % ( mean_squared_error(y, y_pred), r2_score(y, y_pred))


In [375]:
df = load_boston()
df.data    = StandardScaler().fit_transform(df.data)
df.classes = np.array([ int(y > np.mean(df.target)) for y in df.target ])

## Gradient Descent

In [3]:
def least_square_gradient(x, y, B, verbose=False):
    loss = np.dot(x, B) - y 
    if(verbose): print "MSE: ", np.mean(loss**2)
    return np.dot(np.transpose(x), loss)

def gradient_descent(x, y, gradient_func, learning_rate=.00001, max_iterations=1000):
    coefs  = np.random.normal(size=x.shape[1])
    _coefs = [0] * coefs.shape[0]

    for i in range(max_iterations):
        if np.all(coefs == _coefs): break
        _coefs = coefs
        coefs  = coefs - learning_rate * gradient_func(x, y, coefs)
    
    return coefs

In [18]:
data = StandardScaler().fit_transform(bost.data)

lr = LinearRegression().fit(data, df.target)
print "LR ", evaluate(lr.coef_, data, df.target)

coefs = gradient_descent(data, df.target, least_square_gradient, learning_rate=.0005, max_iterations=1000)
print "Gradient ", evaluate(coefs, data, df.target)

LR  MSE 529.625 / R2 -5.274
Gradient  MSE 529.625 / R2 -5.274


## Maximum likelihood

In [422]:
np.random.normal()

-0.5773603518468153

In [430]:
import scipy as sc

def logit_likelihood(B, X, y, verbose=False):
    eps = 1e-15
    p   = sc.special.expit( np.dot(X,B) )
    p   = np.array( [ min(eps, max(i, 1-eps)) for i in p ] )
    ll  = y*np.log(p) + (1-y)*np.log(1.0-p)
    return log_loss(y, p)

def maximum_likelihood(X, y, likelihood_estimation):
    init_B = np.random.normal(size=X.shape[1])
    init_B = [0] * X.shape[1]
    result = sc.optimize.minimize(logit_likelihood, x0=init_B, args=(X,y), method='Powell', options={ 'disp': True })
    return result.x 

In [429]:

lr = LogisticRegression().fit(df.data, df.classes)
print "LR ", evaluate(lr.coef_[0], df.data, df.classes, logit=True)

coefs = maximum_likelihood(df.data, df.classes, logit_likelihood)
print "ML ", evaluate(coefs, df.data, df.classes, logit=True)

LR  LL 0.332 / AUC 0.946
Optimization terminated successfully.
         Current function value: 13.600808
         Iterations: 1
         Function evaluations: 176
ML  LL 3.503 / AUC 0.301


## Bayes estimator

## Expectation Maximization

## Markov chain